In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# START

**Imports**

In [ ]:
from scipy.stats import norm
from datetime import datetime

from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb # regression model

# plots
import seaborn as sns
import matplotlib.pyplot as plt

**Read File**

This notebook will work with any csv file in the '100,000 UK Used Car' data set. XGBoost always gives the best score; it will give an r^2 score of 0.90 or greater. Change the csv file and hit 'Run All' to see different results. The cclass file does not have tax or mpg as features and performs the worst at an r2 score of 0.90. The best score is with the vw.csv file with an r2 of ~0.9642.

In [ ]:
ford = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/ford.csv')
ford.info()

In [ ]:
if 'tax(£)' in ford.columns:
    ford.rename({'tax(£)': 'tax'}, axis='columns')

Distribution of the response variable:

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
sns.distplot(ford['price'], fit=norm, ax=ax1)
sns.distplot(np.log(ford['price']), fit=norm, ax=ax2)
plt.show()

Transform the price to get a normal distribution:

In [ ]:
ford['price'] = np.log(ford['price'])

In [ ]:
def price_plot(feature, target='price', width=8, height=7, flip=False):
    if feature in ford.columns:
        plt.figure(figsize=(width, height))
        price_order = ford.groupby(feature)[target].mean().sort_values(ascending=False).index.values
        if flip:
            sns.boxplot(data=ford, y=target, x=feature, order=price_order)
        else:
            sns.boxplot(data=ford, x=target, y=feature, order=price_order)
        plt.title('Price of Ford vehicles by %s' %feature)

# Eploratory Data Analysis (EDA)

**Model**

In [ ]:
ford['model'] = ford['model'].str.strip()
price_plot('model')

**Year**

In [ ]:
price_plot('year', flip=True, height=5, width=12)

**Transmission**

In [ ]:
price_plot('transmission', flip=True, width=5, height=4)

**Mileage**

In [ ]:
sns.lmplot(data=ford, x='mileage', y='price')

Take the square root of the mileage to get a normal distribution:

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
sns.distplot(ford['mileage'], fit=norm, ax=ax1)
sns.distplot(np.sqrt(ford['mileage']), fit=norm, ax=ax2)
plt.show()

In [ ]:
ford['mileage'] = np.sqrt(ford['mileage'])
sns.lmplot(data=ford, x='mileage', y='price')

**Fuel Type**

In [ ]:
price_plot('fuelType', flip=True, width=5, height=4)

**Tax**

In [ ]:
price_plot('tax', flip=True, width=10, height=5)

**Miles per gallon (mpg)**

In [ ]:
price_plot('mpg', flip=True, width=30, height=5)

In [ ]:
if 'mpg' in ford.columns:
    sns.lmplot(data=ford, x='mpg', y='price')

**Engine Size**

In [ ]:
price_plot('engineSize', flip=True, width=10, height=5)

In [ ]:
sns.lmplot(data=ford, x='engineSize', y='price')

# Encoding

The categorical variables need to be converted into numerical variables for training.

In [ ]:
ford.info()

In [ ]:
to_one_hot = ['model', 'transmission', 'fuelType']

temp = ford.copy()
temp = pd.concat([temp, pd.get_dummies(temp[to_one_hot])], axis=1)
temp.drop(to_one_hot, axis=1, inplace=True)
ford = temp
ford.head()

# Predictions

In [ ]:
y = ford.price
X = ford.drop(columns='price', axis=1)
X.head(10)

In [ ]:
pd.DataFrame(np.exp(y)).head(10).T

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

def validation_predictions(model, name):
    new_model = model.fit(X_train, y_train)
    pred = np.exp(new_model.predict(X_valid))
    print("============= %s and Shuffle Split =============" %name)
    print("Accuracy: %f" %(r2_score(np.exp(y_valid), pred)))
    print("RMSE: %f" %(np.sqrt(mean_squared_error(np.exp(y_valid), pred))))
    print("MAE: %f" %(mean_absolute_error(np.exp(y_valid), pred)))
    
    new_model = model.fit(X, y)
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    cvs = cross_val_score(new_model, X, y, cv=cv)
    print('Shuffle and cross validate: %s \nAverage: %.4f' %(cvs, cvs.mean()))

In [ ]:
lr_model = LinearRegression()
validation_predictions(lr_model, 'Linear Regression')

In [ ]:
dt_model = DecisionTreeRegressor(random_state=0)
validation_predictions(dt_model, 'Decision Tree Regressor')

In [ ]:
rf_model = RandomForestRegressor(random_state=0)
validation_predictions(rf_model, 'Random Forest Regressor')

In [ ]:
xgb_model = xgb.XGBRegressor()
validation_predictions(xgb_model, 'XGBoost Regressor')